In [1]:
import sys
import os

parent_dir = os.path.abspath('../')  
sys.path.append(parent_dir)

from get_benchmark_results_utils import *

In [3]:
log_path = "../logs/ours/our_logs_with_contributions_111_4.log"
our_results = get_contribution_results_from_logs(log_path)

(None, None) None
('safe', 0.007266) GreaterEqThan@comparators@circomlib_16.circom
('safe', 0.000318) BabyCheck@babyjub@circomlib.circom
('safe', 0.194688) Num2BitsNeg@bitify@circomlib_256.circom
('safe', 0.009747) GreaterEqThan@comparators@circomlib_32.circom
('safe', 0.005931) Num2BitsNeg@bitify@circomlib_16.circom
('safe', 0.00057) Num2Bits@bitify@circomlib_1.circom
('safe', 3.081286) T1@t1@circomlib.circom
('unknown', 30.061604976654053) Main@main@circomlib.circom
('safe', 0.01426) Bits2Num@bitify@circomlib_256.circom
('unknown', 30.002083778381348) BabyPbk@babyjub@circomlib.circom
('safe', 0.00064) Sigma@poseidon@circomlib.circom
('safe', 2.953194) EdDSAPoseidonVerifier@eddsaposeidon@circomlib.circom
('safe', 8.589725) BabyAdd@babyjub@circomlib.circom
('safe', 16.320074) Point2Bits_Strict@pointbits@circomlib.circom
('safe', 0.000825) Bits2Num@bitify@circomlib_32.circom
('safe', 0.021195) Mux4@mux4@circomlib.circom
('safe', 0.011912) GreaterThan@comparators@circomlib_32.circom
('un

In [4]:
def analyze_contribution_types(our_results):
    contribution_type_counts = {}
    bpg_time = list()
    bpg_node_count = list()
    num_assumptions_count = list()

    total_files = len(our_results)

    for filename, data in our_results.items():
        contributions = data['contributions']
        counted_contributions = set()
        
        for contribution_type in contributions:
            assert contribution_type not in counted_contributions
            
            if contribution_type == 'bpg_time_records':
                bpg_time.extend(contributions[contribution_type])
                
            if contribution_type == 'bpg_nodes':
                bpg_node_count.append(contributions[contribution_type])
            
            if contribution_type == 'add_edge_to_assumptions':
                num_assumptions_count.append(contributions[contribution_type])
                

            if contribution_type in ['simple_replacement_expression_CONS_propogation', 'bpg_time_records', 'NO_Assumptions_raw_SMT_UNDER']:
                continue
                
            if contribution_type == 'BPG_binary':
                contribution_type = 'BPG_solved'
            if contribution_type == 'BPG_constrained_both_solved':
                if 'BPG_binary' not in contributions:
                    contribution_type = 'BPG_solved'
                else:
                    continue
                    
            if contribution_type == 'simple_replacement_expression':
                contribution_type = 'simple_replacement'
            if contribution_type == 'simple_replacement_constant':
                if 'simple_replacement_expression' not in contributions:
                    contribution_type = 'simple_replacement'
                else:
                    continue    
                    
             
            if contribution_type == 'Assumptions_via_grobner_UNDER':
                contribution_type = 'Assumption_solved'
            if contribution_type == 'Assumptions_grobner_early_conflict_cons':
                if 'Assumptions_via_grobner_UNDER' not in contributions:
                    contribution_type = 'Assumption_solved'
                else:
                    continue 
                    
                    
                    
            if contribution_type == 'bpg_nodes':
                contribution_type = 'has_BPG_nodes'
                 
            if contribution_type == 'bpg_edges':
                contribution_type = 'has_BPG_edges'
            
            if contribution_type in contribution_type_counts:
                contribution_type_counts[contribution_type] += 1
            else:
                contribution_type_counts[contribution_type] = 1
            counted_contributions.add(contribution_type)
            
            

    contribution_type_percentages = {}
    for contribution_type, count in contribution_type_counts.items():
        contribution_type_percentages[contribution_type] = (count / total_files) * 100

    return contribution_type_counts, contribution_type_percentages, bpg_time, bpg_node_count, num_assumptions_count

contribution_counts, contribution_percentages, bpg_time, bpg_node_count, num_assumptions_count = analyze_contribution_types(our_results)

print("Contribution Counts:")
for contribution, count in contribution_counts.items():
    print(f"{contribution}: {count}")
    
print("\n clause_learning percentage out of assumption phases: ")
print(contribution_counts["clause_learning"]/contribution_counts["DO_ASSUMPTION_PHASE"] *100)

print("\n Assumption_solved percentage out of assumption phases: ")
print(contribution_counts["Assumption_solved"]/contribution_counts["DO_ASSUMPTION_PHASE"] *100)
    

print("\nContribution Percentages:")
for contribution, percentage in contribution_percentages.items():
    print(f"{contribution}: {percentage:.2f}%")

Contribution Counts:
simple_replacement: 96
has_BPG_edges: 150
has_BPG_nodes: 150
multi_con_to_single_uncon: 141
BPG_solved: 59
binary_expression: 46
single_to_double: 138
add_edge_to_assumptions: 58
DO_ASSUMPTION_PHASE: 38
assumption_smt_count: 34
clause_learning: 12
Assumption_solved: 15
solved_assumption_smt_count: 8
unsolved_assumption_smt_count: 8

 clause_learning percentage out of assumption phases: 
31.57894736842105

 Assumption_solved percentage out of assumption phases: 
39.473684210526315

Contribution Percentages:
simple_replacement: 54.55%
has_BPG_edges: 85.23%
has_BPG_nodes: 85.23%
multi_con_to_single_uncon: 80.11%
BPG_solved: 33.52%
binary_expression: 26.14%
single_to_double: 78.41%
add_edge_to_assumptions: 32.95%
DO_ASSUMPTION_PHASE: 21.59%
assumption_smt_count: 19.32%
clause_learning: 6.82%
Assumption_solved: 8.52%
solved_assumption_smt_count: 4.55%
unsolved_assumption_smt_count: 4.55%


In [5]:
full_stats = load_data_structure("../logs/saved_ds/benchmark_stats.yaml")
all_circuit_stats = {}
for type_bm in full_stats:
    for k in full_stats[type_bm]:
        assert k not in all_circuit_stats
        all_circuit_stats[k] = full_stats[type_bm][k]
        all_circuit_stats[k]["type_bench"] = type_bm

In [6]:
all_circuit_stats

{'AND@gates@circomlib.circom': {'category': 'small',
  'linear constraints': 0,
  'non-linear constraints': 1,
  'private inputs': 0,
  'private outputs': 0,
  'public outputs': 1,
  'total constraints': 1,
  'total inputs': 2,
  'total outputs': 1,
  'wires': 4,
  'witness': 1,
  'type_bench': 'core'},
 'AliasCheck@aliascheck@circomlib.circom': {'category': 'medium',
  'linear constraints': 259,
  'non-linear constraints': 262,
  'private inputs': 0,
  'private outputs': 0,
  'public outputs': 0,
  'total constraints': 521,
  'total inputs': 254,
  'total outputs': 0,
  'wires': 774,
  'witness': 520,
  'type_bench': 'core'},
 'BabyAdd@babyjub@circomlib.circom': {'category': 'small',
  'linear constraints': 0,
  'non-linear constraints': 6,
  'private inputs': 0,
  'private outputs': 0,
  'public outputs': 2,
  'total constraints': 6,
  'total inputs': 4,
  'total outputs': 2,
  'wires': 11,
  'witness': 5,
  'type_bench': 'core'},
 'BabyCheck@babyjub@circomlib.circom': {'category': '

In [ ]:
Empirical complexity:

In [7]:
c = 0
for k in our_results:
    if "bpg_nodes" in our_results[k]['contributions']:
        c += our_results[k]['contributions']['bpg_nodes']
print(c)

c = 0
for k in our_results:
    if "bpg_edges" in our_results[k]['contributions']:
        c += our_results[k]['contributions']['bpg_edges']
print(c)

252598
2882381


In [8]:
for size in ['small', 'medium', 'large']:
    l = []
    for k in our_results:
        if "bpg_edges" in our_results[k]['contributions']:
            if all_circuit_stats[k]['category'] == size:
    #             print(our_results[k]['contributions']['bpg_edges'] / all_circuit_stats[k]['wires'])
                l.append(our_results[k]['contributions']['bpg_nodes'] / all_circuit_stats[k]['wires'])
    print(size, sum(l)/len(l))

small 2.6988097060829923
medium 3.0973616660454746
large 2.6804125908786016


In [9]:
for size in ['small', 'medium', 'large']:
    l = []
    for k in our_results:
        if "bpg_edges" in our_results[k]['contributions']:
            c += our_results[k]['contributions']['bpg_edges']
            if all_circuit_stats[k]['category'] == size:
    #             print(our_results[k]['contributions']['bpg_edges'] / all_circuit_stats[k]['wires'])
                l.append(our_results[k]['contributions']['bpg_edges'] / all_circuit_stats[k]['wires'])
    print(size, sum(l)/len(l))

small 18.797899447415972
medium 23.783118919162057
large 20.605719269697637


In [10]:
for size in ['small', 'medium', 'large']:
    l = []
    for k in our_results:
        if "bpg_time_records" in our_results[k]['contributions']:
            if all_circuit_stats[k]['category'] == size:
    #             print(our_results[k]['contributions']['bpg_edges'] / all_circuit_stats[k]['wires'])
                cl = list(our_results[k]['contributions']['bpg_time_records'])
                l.append((sum(cl)) / all_circuit_stats[k]['wires'])
    print(size, sum(l)/len(l) * 1000)

small 0.23951096280355497
medium 0.5974480240809721
large 0.6700740344194472


In [11]:
for size in ['small', 'medium', 'large']:
    l = []
    for k in our_results:
        if all_circuit_stats[k]['category'] == size:
            l.append(all_circuit_stats[k]['wires'])
    print(size, sum(l)/len(l))

small 35.526785714285715
medium 495.75
large 27587.4375


In [ ]:
SMT success rate:

In [12]:
total_count = 0
total_not_sure = 0
for k in our_results:
    total = our_results[k]['contributions'].get('assumption_smt_count', 0)
    solved = our_results[k]['contributions'].get('solved_assumption_smt_count', 0)
    under_solved = our_results[k]['contributions'].get('unsolved_assumption_smt_count', 0)
    not_sure = total - solved - under_solved
    if total != 0:
        print(total, solved, under_solved, not_sure)
        total_count += total
        total_not_sure += not_sure

2 2 0 0
1 0 0 1
1 0 0 1
1 0 0 1
1 0 0 1
2 1 1 0
1 0 1 0
2 2 0 0
1 0 0 1
1 0 0 1
1 0 0 1
1 0 0 1
1 0 1 0
1 0 0 1
1 0 0 1
2 2 0 0
1 0 0 1
1 0 0 1
1 0 0 1
1 0 0 1
1 0 0 1
2 2 0 0
1 0 0 1
2 2 0 0
2 0 0 2
1 0 1 0
1 0 0 1
1 0 0 1
1 0 1 0
1 0 1 0
1 0 0 1
1 0 0 1
2 1 1 0
2 1 1 0


In [13]:
total_count, total_not_sure

(43, 22)

In [14]:
1 - 22/43

0.4883720930232558

In [15]:
48.84 / 2.68

18.223880597014926